In [ ]:
pip install mysql-connector-python

In [256]:
import mysql.connector as connector
from mysql.connector import errorcode

try:
    connection = connector.connect(user="root", password="", port=3306, host="localhost")
    print ("Connection to the database is established!")
except connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Connection user or password are incorrect")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print("Error code: ", err.errno)
        print("Error message: ", err.msg)

cursor = connection.cursor()

cursor.execute("""DROP SCHEMA IF EXISTS little_lemon_db;""")
cursor.execute("""CREATE SCHEMA IF NOT EXISTS little_lemon_db;""")
cursor.execute("""USE little_lemon_db;""")

print('The actual database is', connection.database)

Connection to the database is established!
The actual database is little_lemon_db


In [257]:
# Creating tables

# Create Customers table
create_customers_table_query = """
CREATE TABLE IF NOT EXISTS Customers (
  CustomerID INT NOT NULL,
  FullName VARCHAR(45) NOT NULL,
  ContactNumber VARCHAR(45) NOT NULL,
  Email VARCHAR(45) NOT NULL,
  PRIMARY KEY (CustomerID));
"""

# Create MenuItems table
create_menu_items_table_query = """
CREATE TABLE IF NOT EXISTS MenuItems (
  MenuItemID INT NOT NULL,
  CourseName VARCHAR(45) NOT NULL,
  StarterName VARCHAR(45) NOT NULL,
  DessertName VARCHAR(45) NOT NULL,
  PRIMARY KEY (MenuItemID));
"""


# Create Menus table
create_menus_table_query = """
CREATE TABLE IF NOT EXISTS Menus (
  MenuID INT NOT NULL,
  MenuName VARCHAR(45) NOT NULL,
  Cousine VARCHAR(45) NOT NULL,
  MenuItemID INT NOT NULL,
  PRIMARY KEY (MenuID));
"""



# Create Orders table
create_orders_table_query = """
CREATE TABLE IF NOT EXISTS Orders (
  OrderID INT NOT NULL,
  Quantity INT NOT NULL,
  TotalCost DECIMAL(10,2) NOT NULL,
  CustomerID INT NOT NULL,
  MenuID INT NOT NULL,
  PRIMARY KEY (OrderID));
"""

cursor.execute(create_customers_table_query)
print("Customers table is created")

cursor.execute(create_menu_items_table_query)
print("MenuItems table is created")

cursor.execute(create_menus_table_query)
print("Menus table is created")

cursor.execute(create_orders_table_query)
print("Orders table is created")
print()


# ---------------------------------------------------------------------
# Inserting data into the tables

# Inserting into Customers
insert_customers_table_query = """
INSERT INTO Customers (CustomerID, FullName, ContactNumber, Email)
VALUES
(1, "Laney Fadden", "+491634556622", "laney@fadden.com"),
(2, "Giacopo Bramich", "+492345647878", "giacopo@bramich.com"),
(3, "Lia Bonar", "+796538453452", "lia@bonar.com"),
(4, "Merrill Baudon", "+18250081324", "merrill@baudon.com"),
(5, "Tasia Fautly", "+163544779812", "tasia@fautly.com");
"""

# Inserting into MenuItems
insert_menu_items_table_query = """
INSERT INTO MenuItems (MenuItemID, CourseName, StarterName, DessertName)
VALUES
(1, "Greek salad", "Olives", "Greek yoghurt"),
(2, "Bean soup", "Flatbread", "Ice cream"),
(3, "Pizza", "Minestrone", "Cheesecake"),
(4, "Carbonara", "Tomato bread", "Affogato"),
(5, "Kabasa", "Falafel", "Turkish yoghurt");
"""

# Inserting into Menus table
insert_menus_table_query = """
INSERT INTO Menus (MenuID, MenuName, Cousine, MenuItemID)
VALUES
(1, "Summer", "Greek", 1),
(2, "Spring", "Italian", 2),
(3, "Winter", "Turkish", 3),
(4, "Fall", "Italian", 4),
(5, "Seasonal", "Turkish", 5);
"""

# Inserting into Orders table
insert_orders_table_query = """
INSERT INTO Orders (OrderID, Quantity, TotalCost, CustomerID, MenuID)
  VALUES
  (1, 3, 135.50, 1, 1),
  (2, 2, 192.00, 2, 2),
  (3, 1, 23.50, 3, 3),
  (4, 5, 67.50, 4, 4),
  (5, 3, 90.00, 5, 5);
"""

cursor.execute(insert_customers_table_query)
connection.commit()

cursor.execute(insert_menu_items_table_query)
connection.commit()

cursor.execute(insert_menus_table_query)
connection.commit()

cursor.execute(insert_orders_table_query)
connection.commit()

show_tables_query = """SHOW TABLES"""
cursor.execute(show_tables_query)
print("Tables in the DB:")
for table in cursor:
    print(table)

Customers table is created
MenuItems table is created
Menus table is created
Orders table is created

Tables in the DB:
('Customers',)
('MenuItems',)
('Menus',)
('Orders',)


In [258]:
# Create a virtual table to summarize data
# Task 1. Creating OrderView

cursor.execute("""DROP VIEW IF EXISTS OrderView""")
connection.commit()

create_order_view_query = """
CREATE VIEW OrderView AS 
SELECT OrderID, Quantity, FORMAT(TotalCost, 2) AS Cost
FROM Orders;
"""
cursor.execute(create_order_view_query)
connection.commit()

# Fetch and print the view results
cursor.execute("""SELECT * FROM OrderView""")
results = cursor.fetchall()
cols = cursor.column_names

print(cols)
for result in results:
    print(result)
print()

# ---------------------------------------------------------------------
# Task 2. Creating a summary table from 

summary_table_query = """
SELECT 
Customers.CustomerID,
Customers.FullName,
Orders.OrderID,
FORMAT(Orders.TotalCost, 2) AS Cost,
Menus.MenuName,
MenuItems.CourseName
FROM Orders
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID
INNER JOIN Menus ON Orders.MenuID = Menus.MenuID
INNER JOIN MenuItems ON Menus.MenuItemID = MenuItems.MenuItemID;
"""

# Fetch and print the view results
cursor.execute(summary_table_query)
results = cursor.fetchall()
cols = cursor.column_names

print(cols)
for result in results:
    print(result)
print()

# ---------------------------------------------------------------------
# Task 3. Creating a table with all menu items for which more than 2 orders have been placed

select_menu_items_query = """
SELECT MenuName FROM Menus
WHERE MenuID = ANY (SELECT Quantity FROM Orders WHERE Quantity > 2);
"""

# Fetch and print the view results
cursor.execute(select_menu_items_query)
results = cursor.fetchall()
cols = cursor.column_names

print(cols)
for result in results:
    print(result)
print()

('OrderID', 'Quantity', 'Cost')
(1, 3, '135.50')
(2, 2, '192.00')
(3, 1, '23.50')
(4, 5, '67.50')
(5, 3, '90.00')

('CustomerID', 'FullName', 'OrderID', 'Cost', 'MenuName', 'CourseName')
(1, 'Laney Fadden', 1, '135.50', 'Summer', 'Greek salad')
(2, 'Giacopo Bramich', 2, '192.00', 'Spring', 'Bean soup')
(3, 'Lia Bonar', 3, '23.50', 'Winter', 'Pizza')
(4, 'Merrill Baudon', 4, '67.50', 'Fall', 'Carbonara')
(5, 'Tasia Fautly', 5, '90.00', 'Seasonal', 'Kabasa')

('MenuName',)
('Winter',)
('Seasonal',)



In [259]:
# Create optimized queries to manage and analyze data
# Task 1. Creating a procedure that displays the maximum ordered quantity in the Orders table

cursor.execute("""DROP PROCEDURE IF EXISTS GetMaxQuantity""")
connection.commit()

create_procedure_get_max_quantity_query = """
    CREATE PROCEDURE GetMaxQuantity()
    SELECT MAX(Quantity) AS "Max Quantity in Order" 
    	FROM Orders;
"""

cursor.execute(create_procedure_get_max_quantity_query)
cursor.callproc("GetMaxQuantity")

results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

print(columns)

for data in dataset:
    print(data)
print()

# ---------------------------------------------------------------------
# Task 2. Creating a prepare statement

prepare_statement_query = """
    PREPARE GetOrderDetail FROM
    "SELECT OrderID, Quantity, TotalCost
    FROM Orders
    WHERE OrderID = ?";
    """

cursor.execute(prepare_statement_query)
order_id = 2

cursor.execute("EXECUTE GetOrderDetail USING @id", {'id': order_id})

# Fetch and print the results
results = cursor.fetchall()
cols = cursor.column_names

print(cols)
for result in results:
    print(result)
cursor.execute("DEALLOCATE PREPARE GetOrderDetail")
print()

# ---------------------------------------------------------------------
# Task 3. Creating a stored procedure CancelOrder

cursor.execute("DROP PROCEDURE IF EXISTS CancelOrder;")

cancel_order_query = """
CREATE PROCEDURE CancelOrder(IN OrderIDParam INT)
BEGIN
	DELETE FROM Orders WHERE OrderID = OrderIDParam;
	SELECT CONCAT("Order ", OrderIDParam, " is cancelled!") AS Confirmation;
END
"""
order_id_to_cancel = 5

cursor.execute(cancel_order_query)
cursor.callproc("CancelOrder", (order_id_to_cancel,))
connection.commit()

results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

print(columns)

for data in dataset:
    print(data)
print()

['Max Quantity in Order']
(5,)

('OrderID', 'Quantity', 'TotalCost')

['Confirmation']
('Order 5 is cancelled!',)



In [260]:
# Create SQL queries to check available bookings based on user input
# Task 1: Creating bookings table
create_bookings_table_query = """
CREATE TABLE IF NOT EXISTS Bookings (
  BookingID INT NOT NULL AUTO_INCREMENT,
  TableNumber INT NOT NULL,
  BookingDate Date NOT NULL,
  CustomerID INT,
  PRIMARY KEY (BookingID));
"""

cursor.execute(create_bookings_table_query)
connection.commit()

show_tables_query = """SHOW TABLES"""
cursor.execute(show_tables_query)
print("Tables in the DB:")
for table in cursor:
    print(table)
print()

# Inserting into Bookings table
insert_bookings_table_query = """
INSERT INTO Bookings (BookingID, BookingDate, TableNumber, CustomerID)
  VALUES
  (1, "2022-10-10", 5, 1),
  (2, "2022-11-12", 3, 3),
  (3, "2022-10-11", 2, 2),
  (4, "2022-10-13", 2, 1);
"""

cursor.execute(insert_bookings_table_query)
connection.commit()

# ---------------------------------------------------------------------
# Task 2: Create a stored procedure called CheckBooking to check whether a table in the restaurant is already booked

cursor.execute("DROP PROCEDURE IF EXISTS CheckBooking;")

create_check_booking_procedure_query = """
CREATE PROCEDURE CheckBooking(IN booking_date DATE, IN table_number INT)
BEGIN
DECLARE booking_count INT;
    SET booking_count = (
        SELECT COUNT(*) FROM Bookings 
        WHERE BookingDate = booking_date AND TableNumber = table_number
    );
    
    IF booking_count > 0 THEN
        SELECT CONCAT("Table ", table_number, " is already booked.") AS "Booking status";
    ELSE 
        SELECT CONCAT("Table ", table_number, " is not booked.") AS "Booking status";
    END IF;
END
"""

booking_date = "2022-11-12"
table_number = 3

cursor.execute(create_check_booking_procedure_query)
cursor.callproc("CheckBooking", (booking_date, table_number))
connection.commit()

results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

print("Check the booking:")
print(columns)

for data in dataset:
    print(data)
print()

# ---------------------------------------------------------------------
# Task 3: Create a stored procedure called CheckBooking  to verify a booking, 
# and decline any reservations for tables that are already booked under another name

cursor.execute("DROP PROCEDURE IF EXISTS AddValidBooking;")

# Create the AddValidBooking procedure
create_add_valid_booking_procedure_query = """
CREATE PROCEDURE AddValidBooking(IN booking_date DATE, IN table_number INT)
BEGIN
    DECLARE booking_count INT;
    DECLARE booking_exists INT DEFAULT 0;
    
    START TRANSACTION;
    
    SELECT COUNT(*) INTO booking_count FROM Bookings 
    WHERE BookingDate = booking_date AND TableNumber = table_number;
    
    IF booking_count > 0 THEN
        SET booking_exists = 1;
    ELSE
        INSERT INTO Bookings (BookingDate, TableNumber, CustomerID)
        VALUES (booking_date, table_number, 1);
    END IF;
    
    IF booking_exists = 1 THEN
        ROLLBACK;
        SELECT CONCAT("Table ", table_number, " is already booked - cancelling booking") AS BookingStatus;
    ELSE
        COMMIT;
        SELECT CONCAT("Table ", table_number, " is not booked - new booking added") AS BookingStatus;
    END IF;
    
END
"""

cursor.execute(create_add_valid_booking_procedure_query)

# Call the AddValidBooking procedure
booking_date = "2022-11-12"
table_number = 6

cursor.callproc("AddValidBooking", (booking_date, table_number))
connection.commit()

# Fetch and print the result
results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

print("Add valid booking:")
print(columns)

for data in dataset:
    print(data)
print()

Tables in the DB:
('Bookings',)
('Customers',)
('MenuItems',)
('Menus',)
('Orders',)
('orderview',)

Check the booking:
['Booking status']
('Table 3 is already booked.',)

Add valid booking:
['BookingStatus']
('Table 6 is not booked - new booking added',)



In [261]:
# Create SQL queries to add and update bookings
# Task 1: Create a new procedure called AddBooking to add a new table booking record

cursor.execute("DROP PROCEDURE IF EXISTS AddBooking;")

# Create the AddValidBooking procedure
create_add_booking_procedure_query = """
CREATE PROCEDURE AddBooking(IN booking_id INT, IN booking_date DATE, IN table_number INT, IN customer_id INT)
BEGIN
    INSERT INTO Bookings (BookingID, BookingDate, TableNumber, CustomerID)
    VALUES (booking_id, booking_date, table_number, customer_id);
    SELECT "New booking added" AS "Confirmation";
END
"""

cursor.execute(create_add_booking_procedure_query)

# Call the AddBooking procedure
booking_id = 9
booking_date = "2022-11-12"
table_number = 6
customer_id = 4

cursor.callproc("AddBooking", (booking_id, booking_date, table_number, customer_id))
connection.commit()

# Fetch and print the result
results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

print("Add booking:")
print(columns)

for data in dataset:
    print(data)
print()

# ---------------------------------------------------------------------
# Task 2: Create a new procedure called UpdateBooking to update booking record

cursor.execute("DROP PROCEDURE IF EXISTS UpdateBooking;")

# Create the UpdateBooking procedure
create_update_booking_procedure_query = """
CREATE PROCEDURE UpdateBooking(IN booking_id INT, IN booking_date DATE)
BEGIN
    UPDATE Bookings
    SET BookingDate = booking_date
    WHERE BookingID = booking_id;
    
    SELECT CONCAT("Booking ", booking_id, " updated") AS "Confirmation";
END
"""

cursor.execute(create_update_booking_procedure_query)

# Call the UpdateBooking procedure
booking_id = 9
booking_date = "2022-11-13"

cursor.callproc("UpdateBooking", (booking_id, booking_date))
connection.commit()

# Fetch and print the result
results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

print("Update booking:")
print(columns)

for data in dataset:
    print(data)
print()


# ---------------------------------------------------------------------
# Task 3: Create a new procedure called CancelBooking to cancel the booking record

cursor.execute("DROP PROCEDURE IF EXISTS CancelBooking;")

# Create the CancelBooking procedure
create_cancel_booking_procedure_query = """
CREATE PROCEDURE CancelBooking(IN booking_id INT)
BEGIN
    DELETE FROM Bookings
    WHERE BookingID = booking_id;
    
    SELECT CONCAT("Booking ", booking_id, " cancelled") AS "Confirmation";
END
"""

cursor.execute(create_cancel_booking_procedure_query)

# Call the CancelBooking procedure
booking_id = 9

cursor.callproc("CancelBooking", (booking_id, ))
connection.commit()

# Fetch and print the result
results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

print("Cancel booking:")
print(columns)

for data in dataset:
    print(data)
print()

Add booking:
['Confirmation']
('New booking added',)

Update booking:
['Confirmation']
('Booking 9 updated',)

Cancel booking:
['Confirmation']
('Booking 9 cancelled',)



In [262]:
# Add query functions
# Task 2: Show all tables
show_tables_query = """SHOW TABLES"""
cursor.execute(show_tables_query)
print("Tables in the DB:")
for table in cursor:
    print(table)
print()

# ---------------------------------------------------------------------
# Task 3: Query with table JOIN
customer_data_query = """
SELECT 
    Customers.FullName, 
    Customers.ContactNumber, 
    Customers.Email
FROM Customers INNER JOIN Orders ON Customers.CustomerID = Orders.CustomerID
WHERE Orders.TotalCost > 60;
"""

# Fetch and print the view results
cursor.execute(customer_data_query)
results = cursor.fetchall()
cols = cursor.column_names

print("Details for every customer that has placed an order greater than $60:")
print(cols)
for result in results:
    print(result)
print()

Tables in the DB:
('Bookings',)
('Customers',)
('MenuItems',)
('Menus',)
('Orders',)
('orderview',)

Details for every customer that has placed an order greater than $60:
('FullName', 'ContactNumber', 'Email')
('Laney Fadden', '+491634556622', 'laney@fadden.com')
('Giacopo Bramich', '+492345647878', 'giacopo@bramich.com')
('Merrill Baudon', '+18250081324', 'merrill@baudon.com')



In [263]:
if connection.is_connected():
    cursor.close()
    print("The cursor is closed.")
    connection.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")

The cursor is closed.
MySQL connection is closed.
